In [61]:
def predict(input_string: str):
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  from tensorflow.keras.models import load_model
  lstm = load_model("lstm.h5", custom_objects={'mse': tf.keras.losses.mse}) # Pass mse as a custom object during model loading
  import joblib
  loaded_results = joblib.load("sarimax_model.joblib")
  date1=pd.to_datetime("2025-03-03")
  date2=pd.to_datetime(input_string)
  delta_days = (date2 - date1).days
  date4=pd.to_datetime("2025-02-02")
  delta_days2=(date2-date4).days+1
  dt=pd.read_csv('dt.csv')
  from datetime import timedelta
  def predict_future_regressor_values(model, df, seq_length, n_future):
      last_sequence = df['weighted_sentiment'].values[-seq_length:]
      current_seq = last_sequence.reshape((1, seq_length, 1))
      last_date = pd.to_datetime(df['date'].iloc[-1])
      future_dates = [last_date + timedelta(days=i) for i in range(1, n_future+1)]

      preds = []
      for _ in range(n_future):
          next_value = model.predict(current_seq)[0, 0]
          preds.append(next_value)
          current_seq = np.append(current_seq[:,1:,:], [[[next_value]]], axis=1)

      future_df = pd.DataFrame({
          'date': future_dates,
          'weighted_sentiment': preds
      })
      return future_df
  seq_length=14
  future_predictions = predict_future_regressor_values(lstm,dt, seq_length, n_future=delta_days)
  dt2=dt.iloc[-30:,:]
  dt_combined = pd.concat([dt2,future_predictions ], ignore_index=True)
  dt['date']=pd.to_datetime(dt['date'])
  future_predictions['date']=pd.to_datetime(future_predictions['date'])
  dt2['date'] = pd.to_datetime(dt2['date'])
  future_predictions['date'] = pd.to_datetime(future_predictions['date'])
  dt_combined=dt_combined['weighted_sentiment']
  dt_combined=dt_combined.to_numpy()
  future_dates = pd.date_range(start=date4, periods=delta_days2, freq='D')
  future_exog = pd.DataFrame({
      'weighted_sentiment':dt_combined
  }, index=future_dates)
  forecast_obj = loaded_results.get_forecast(steps=delta_days2, exog=future_exog)
  forecast_mean = forecast_obj.predicted_mean
  forecast_ci = forecast_obj.conf_int()
  forecast_mean=pd.DataFrame(forecast_mean)
  forecast_mean['date']=future_dates
  forecast_ci=pd.DataFrame(forecast_ci)
  forecast_ci['date']=future_dates
  return forecast_mean,forecast_ci

In [66]:
import dill
with open("predict.pkl", "wb") as f:
    dill.dump(predict, f)